# CSV aus Timo erstellen

1. Ganzen Monat auswählen
2. Projektauswertung "Grobansicht der Zwischensummen" (Sortieren nach: Mitarbeiter, Projekte)
3. Oben rechts "Daten exportieren ("Tabstopp" als Default lassen)

In [ ]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl

import pandas
import requests

In [ ]:
def read_csv(filename):
    my_data = pandas.read_csv(filename, sep='\t', index_col=False)

    return my_data

In [ ]:
input_filename = 'auswertung.csv'
JSESSIONID = ''
start_date = '2021-03-01'
end_date = '2021-03-31'
output = 'out.xlsx'

In [ ]:

cookies = {
    'JSESSIONID': JSESSIONID,
    'logoffUrl': 'https%3A//www.timo24.de/members/login/',
}

headers = {
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7',
}

params = (
    ('reportType', '9'),
    ('timestamp', '1618492945869'),
    ('start', start_date),
    ('end', end_date),
    ('projekt_id', '-1'),
    ('user_id', '-1'),
    ('vorgang_id', '-1'),
    ('kunden_id', '-1'),
    ('ansprechpartner_id', '-1'),
    ('sort_id_1', '5'),
    ('sort_id_2', '3'),
    ('sort_id_3', '-1'),
    ('sort_id_4', '-1'),
    ('kostenstelle', '-1'),
    ('taskcostunit', '-1'),
    ('ansicht_id', '3'),
    ('team_id', '-1'),
    ('typ_id', '-1'),
    ('auftragsnr', ''),
    ('abrechenbar', '0'),
    ('praemienrelevant', '0'),
    ('kostentraegernummer', ''),
    ('with_closed_vorgaenge', '1'),
    ('datev_filter', '0'),
    ('beschvh_id', '-1'),
    ('seperator', 'TAB'),
    ('andere', ''),
    ('format', '1'),
)

response = requests.get('https://836.timo24.de/timo/reportexport', headers=headers, params=params, cookies=cookies)
open(input_filename, 'wb').write(response.content)

In [ ]:
as_dict = read_csv(input_filename)

df = as_dict.filter(['Datum', 'Projekt', 'Mitarbeiter', 'Stunden'], axis=1)


In [ ]:
def calc_hours(row):
    if row['Stunden']:
        hours = int(row['Stunden'].split(':')[0])
        minutes = int(row['Stunden'].split(':')[1])
        return hours + (minutes/60)
    else:
        return 0                                            

df['Stunden_Dezimal'] = df.apply(lambda row: calc_hours(row), axis=1)
df

In [ ]:
df.drop('Stunden', axis=1)

In [ ]:
df2 = df.groupby(['Mitarbeiter','Projekt'])['Stunden_Dezimal'].agg('sum').reset_index()


In [ ]:
df2.to_excel(output, index = False, header=True)

# Kontrolle

Stunden (Output nächste Zelle) mit Timo-Auswertung abgleichen

In [ ]:
df2['Stunden_Dezimal'].agg('sum')